# DIP opdracht 2: Letterfrequenties
Lucas Siekkötter, 1770991
&
Wijnand van Dijk, 1762733 

### Imports

In [1]:
import numpy
import string
import numpy
from numpy.matrixlib.defmatrix import matrix
from collections import Counter

### Code

In [2]:
leestekens = [".", ",", ":", ";", "\'", "\"", "-", "_", "!", "?", "(", ")", "\\"]

abc = list(string.ascii_lowercase)
abc.extend(["_", "%"])

In [3]:
# Mappers
def mapper_get_lines():
    """
    
    """
    res = []
    with open("data/train_eng/Moby_Dick11.txt", "r", encoding="utf8") as f:
        lines = f.readlines()

        for line in lines:
            line = line.strip()
            if line != "":
                res.append(line)
    
    return res

def mapper_replace_punct(text, sign):
    """
    Replace all punctuation with % and replace spaces with _
    @param  sign: leestekens
    @return     : text waar leestekens vervangen zijn
    """
    res = []
    for line in text:
        for i in sign:
            line = line.replace(i, "%")
        
        line = line.replace(" ", "_")
        line = line.lower()
        res.append(line)

    return res


# Filter
def filter_for_loop(txt):
    """
    Get all 2 letter combinations in text
    @param  txt: tekst waat combinaties uit gehaald worden
    @return    : lijst met combinaties in tekst
    """
    combinations = []
    for line in txt:
        for letter in range(0, len(line)):
            try:
                res = line[letter] + line[letter + 1]
                # leesteken + spatie is niet relevant dus die combinaties kunnen eruit. "\n" zitten er ook tussen en die kunnen ook weg
                if res != "%_" and res != "_%" and "\n" not in res: 
                    combinations.append(res)

            except:
                pass
    
    return combinations

# Reduce
def reducer():
    """
    """
    matrix = numpy.zeros((28, 28))

def main():
    """
    Main function
    """
    lines_list = mapper_get_lines()
    updated_text = mapper_replace_punct(lines_list, leestekens)
    combi_list = filter_for_loop(updated_text)
    print(combi_list[0:100])
    #c = Counter(combi_list)
    #print(c)
    # print(updated_text)


main()

['\ufeffi', 'on', 'ne', 'e_', '_m', 'mo', 'or', 'rn', 'ni', 'in', 'ng', 'g%', '_w', 'wh', 'he', 'en', 'n_', '_g', 'gr', 're', 'eg', 'go', 'or', 'r_', '_s', 'sa', 'am', 'ms', 'sa', 'a_', '_w', 'wo', 'ok', 'ke', 'e_', '_f', 'fr', 'ro', 'om', 'm_', '_t', 'tr', 'ro', 'ou', 'ub', 'bl', 'le', 'ed', 'd_', '_d', 'dr', 're', 'ea', 'am', 'ms', 's%', '_h', 'he', 'e_', '_f', 'fo', 'ou', 'un', 'nd', 'hi', 'im', 'ms', 'se', 'el', 'lf', 'f_', '_t', 'tr', 'ra', 'an', 'ns', 'sf', 'fo', 'or', 'rm', 'me', 'ed', 'd_', '_i', 'in', 'n_', '_h', 'hi', 'is', 's_', '_b', 'be', 'ed', 'd_', '_i', 'in', 'nt', 'to', 'o_', '_a']


In [4]:
az = 'abcdefghijklmnopqrstuvwxyz_%'
def mapper(x):
    print(numpy.zeros((28,28)))

